<a href="https://colab.research.google.com/github/danjgmoyo/EE_Notebooks/blob/main/landsat_9_8_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import ee
import geemap

ee.Authenticate()
ee.Initialize(project = "") #use your project ID
Map = geemap.Map()

In [ ]:
mzimba_district = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Mzimba'))

# Checking if the import was successful or not
print(mzimba_district.size().getInfo())

1


In [ ]:
start_date = '2024-04-02'
end_date = '2024-12-02'
ROI = mzimba_district.geometry()
CLOUD_COVER_MAX = 10

In [ ]:
def mask_clouds_l8(image):
    """Cloud masking using Landsat 8 pixel_qa band"""
    qa = image.select('QA_PIXEL')

    # Bitmask for cloud (bit 3) and cloud shadow (bit 4)
    cloud_mask = (1 << 3)
    shadow_mask = (1 << 4)

    mask = qa.bitwiseAnd(cloud_mask).eq(0).And(
        qa.bitwiseAnd(shadow_mask).eq(0)
    )

    return image.updateMask(mask)

def get_landsat8_collection():
    """Returns processed Landsat 8 SR collection"""
    collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') # you can swap lc08 with LC09
        .filterBounds(ROI)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUD_COVER', CLOUD_COVER_MAX))
        .map(mask_clouds_l8)
    )

    # Apply scaling factors and select bands
    optical_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    thermal_bands = ['ST_B10']

    return collection.map(lambda img:
        img.select(optical_bands, ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
          .multiply(0.0000275)  # Convert to surface reflectance
          .addBands(
              img.select(thermal_bands).multiply(0.00341802).add(149.0)
          )  # Convert thermal band to Kelvin
    )

In [ ]:
# Generate median composite
l8_collection = get_landsat8_collection()
median_l8 = l8_collection.median().clip(ROI)

# select first image
l8_image = l8_collection.first()# .clip(ROI)

In [ ]:
Map.centerObject(ROI, 10)

# Add layers
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # Natural color
    'min': 0.2,
    'max': 0.3
}
Map.addLayer(l8_image, vis_params, 'L8 Natural Color')
Map

Map(bottom=558163.0, center=[-11.770983125351538, 33.63769557063868], controls=(WidgetControl(options=['positi…

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=l8_image,
    description='L8_SR_Export',
    folder='landsatGEE',
    fileNamePrefix='Mzimba',
    region=ROI,
    scale=30,  # Landsat 8 native resolution
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={
        'cloudOptimized': True,
        'noData': 0
    },
    maxPixels=1e13
)

task.start()

print(f"""Export initiated! Key features:
- Landsat 8 Surface Reflectance (Collection 2)
- 30m spatial resolution
- Bands included: B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), B6/B7 (SWIR), ST_B10 (Thermal)
- Cloud masking using pixel_qa band
- Thermal band in Kelvin (ST_B10)

Check your Google Drive folder ' in 15-60 minutes.""")

Export initiated! Key features:
- Landsat 8 Surface Reflectance (Collection 2)
- 30m spatial resolution
- Bands included: B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), B6/B7 (SWIR), ST_B10 (Thermal)
- Cloud masking using pixel_qa band
- Thermal band in Kelvin (ST_B10)

Check your Google Drive folder ' in 15-60 minutes.
